### Load dependencies

In [1]:
import os
from azure.cosmos import CosmosClient, PartitionKey
from azure.cosmos.cosmos_client import ThroughputProperties
from dotenv import load_dotenv

load_dotenv("..\.env")


True

### Step 1: Create Cosmos DB database and container with vector support

In [2]:
COSMOS_DB_ENDPOINT = os.getenv('AZURE_COSMOSDB_NOSQL_ENDPOINT')
COSMOS_DB_KEY = os.getenv('AZURE_COSMOSDB_NOSQL_KEY')
DATABASE_NAME = os.getenv('AZURE_COSMOSDB_NOSQL_DATABASE_NAME')
CONTAINER_NAME = os.getenv('AZURE_COSMOSDB_NOSQL_CONTAINER_NAME')
OFFER_THROUGHPUT = 1000

throughput_properties = ThroughputProperties(auto_scale_max_throughput=OFFER_THROUGHPUT)

indexing_policy = {
    "includedPaths": [
        {"path": "/*"},
    ],
    "excludedPaths": [
        {"path": "/\"_etag\"/?"},
        {"path": "/embedding/*"}
    ],
    "vectorIndexes": [
        {
            "path": "/embedding",
            "type": "quantizedFlat"
        }
    ]
}

embedding_policy = {
    "vectorEmbeddings": [        
        {
            "path": "/embedding",
            "dataType": "float32",
            "distanceFunction": "cosine",
            "dimensions": 384
        }
    ]
}

print("Getting database..")
client = CosmosClient(COSMOS_DB_ENDPOINT, COSMOS_DB_KEY)
database = client.get_database_client(DATABASE_NAME)

# Create "products" container
print(f"Creating '{CONTAINER_NAME}' container..")
container = database.create_container_if_not_exists(
    id=CONTAINER_NAME,
    partition_key=PartitionKey(path="/productId"),
    indexing_policy=indexing_policy,
    vector_embedding_policy=embedding_policy,
    offer_throughput=throughput_properties
)
print(f"'{CONTAINER_NAME}' container created.")


Getting database..
Creating 'products' container..
'products' container created.
